In [143]:
import networkx as nx
import pandas as pd
import numpy as np
from utils import load_data,normalize,doublerelu
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
torch.set_printoptions(sci_mode=False)
import time

In [144]:
import warnings
warnings.filterwarnings("ignore") 

In [179]:
cuda = torch.cuda.is_available()
weight_decay = 10e-4
epochs = 1000
seed = 165
hidden = 10
lr = 0.000001

In [146]:
np.random.seed(seed)
torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)

In [147]:
class GNN1Layer(Module):

    def __init__(self, batch_size, in_features, out_features):
        super(GNN1Layer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.batch_size = batch_size

        weight1_eye = torch.FloatTensor(torch.eye(in_features, out_features))
        weight1_eye = weight1_eye.reshape((1, in_features, out_features))
        weight1_eye = weight1_eye.repeat(batch_size, 1, 1)
        self.weight1 = Parameter(weight1_eye)
        self.weight2 = Parameter(torch.zeros(batch_size, in_features, out_features))

    def forward(self, input, adj):
        v1 = torch.bmm(input, self.weight1)
        v2 = torch.bmm(torch.bmm(adj, input), self.weight2)
        output = v1 + v2
        return output

In [148]:
class GNN2Layer(Module):

    def __init__(self, batch_size, in_features, out_features):
        super(GNN2Layer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.batch_size = batch_size
        weight1_eye = torch.FloatTensor(torch.eye(in_features, out_features))
        weight1_eye = weight1_eye.reshape((1, in_features, out_features))
        weight1_eye = weight1_eye.repeat(batch_size, 1, 1)
        weight1_rand = torch.empty(batch_size,in_features,out_features-in_features)
        torch.nn.init.xavier_uniform_(weight1_rand, gain=1.0)
        self.weight1 = Parameter(weight1_eye)
        self.weight2 = Parameter(torch.zeros(batch_size, in_features, out_features))

    def forward(self, input, adj):
        v1 = torch.bmm(input, self.weight1)
        v2 = torch.bmm(torch.bmm(adj, input), self.weight2)
        output = v1 + v2
        return output

In [149]:
class GNN1(nn.Module):

    def __init__(self, batch_size, nfeat, ndim, hidden):
        super(GNN1, self).__init__()

        self.gc1 = GNN1Layer(batch_size, ndim, ndim)

    def forward(self, x, adj):
        x = doublerelu(self.gc1(x, adj))
        x = x/x.sum(axis=2).unsqueeze(2) #normalize st sum = 1
        return x

In [150]:
class GNN2(nn.Module):

    def __init__(self, batch_size, nfeat, ndim, hidden):
        super(GNN2, self).__init__()

        self.gc1 = GNN2Layer(batch_size, ndim, hidden)
        self.gc2 = GNN1Layer(batch_size, hidden, ndim)

    def forward(self, x, adj):
        x = doublerelu(self.gc1(x, adj))
        x = doublerelu(self.gc2(x, adj))
        x = x/x.sum(axis=2).unsqueeze(2) #normalize st sum = 1
        return x

In [184]:
def train(adj,feature,labels):
    
    adj_norm = normalize(adj)
    
    feature = feature - 1
    nb_label = int(max(feature))+1
    features = np.eye(nb_label)[np.array(feature,dtype=int).reshape(1,-1)]
    
    labels = labels - 1
    
    adj = torch.FloatTensor(adj)
    adj_norm = torch.FloatTensor(adj_norm)
    features = torch.FloatTensor(features)
    labels = torch.FloatTensor(labels)
    
    model = GNN1(batch_size=adj.shape[0],
                nfeat=adj.shape[1],
                ndim=nb_label,
                hidden=hidden)
    if cuda:
        model.cuda()
        features = features.cuda()
        adj = adj.cuda()
        adj_norm = adj_norm.cuda()
        labels = labels.cuda()
    
    # Train model
    t_total = time.time()

    optimizer = optim.Adam(model.parameters(),
                           lr=lr, weight_decay=weight_decay)
    
    criterion = nn.CrossEntropyLoss()
    
    
    for epoch in range(epochs):

        t = time.time()
        model.train()
        optimizer.zero_grad()

        output = model(features, adj_norm)
        
        loss = criterion(output[0],labels.reshape(-1).long())

        loss.backward(retain_graph=True)

        optimizer.step()

        if epoch == 0:
            best_loss = loss
            best_output = output
        else:
            if loss < best_loss:
                best_loss = loss
                best_output = output

        if epoch % 100 == 0:
            print('Epoch: {:04d}'.format(epoch + 1),
                  'Loss: {:.8f}'.format(loss.item()),
                  'time: {:.4f}s'.format(time.time() - t))
            
    print("Optimization Finished!")
    print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
    
    return best_loss,output

In [173]:
adj,feature,labels = load_data()
loss, op = train(adj,feature,labels)

Epoch: 0001 Loss: -1.01517057 time: 0.0040s
Epoch: 0101 Loss: -1.01517057 time: 0.0000s
Epoch: 0201 Loss: -1.01517057 time: 0.0040s
Epoch: 0301 Loss: -1.01517057 time: 0.0020s
Epoch: 0401 Loss: -1.01517057 time: 0.0020s
Epoch: 0501 Loss: -1.01517057 time: 0.0020s
Epoch: 0601 Loss: -1.01517057 time: 0.0000s
Epoch: 0701 Loss: -1.01517057 time: 0.0020s
Epoch: 0801 Loss: -1.01517057 time: 0.0000s
Epoch: 0901 Loss: -1.01517057 time: 0.0000s
Optimization Finished!
Total time elapsed: 1.4867s


In [185]:
loss, op = train(adj,feature,labels)

Epoch: 0001 Loss: 1.01517057 time: 0.0000s
Epoch: 0101 Loss: 1.01517057 time: 0.0000s
Epoch: 0201 Loss: 1.01517057 time: 0.0000s
Epoch: 0301 Loss: 1.01517057 time: 0.0000s
Epoch: 0401 Loss: 1.01517057 time: 0.0000s
Epoch: 0501 Loss: 1.01517057 time: 0.0000s
Epoch: 0601 Loss: 1.01517057 time: 0.0000s
Epoch: 0701 Loss: 1.01517057 time: 0.0000s
Epoch: 0801 Loss: 1.01517057 time: 0.0000s
Epoch: 0901 Loss: 1.01517057 time: 0.0000s
Optimization Finished!
Total time elapsed: 2.7589s


In [186]:
op

tensor([[[0., 1., 0., 0., 0.],
         [0., 1., 0., 0., 0.],
         [0., 1., 0., 0., 0.],
         ...,
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 1.]]], device='cuda:0', grad_fn=<DivBackward0>)